# Loading data from Datawarehouse on Google Cloud Big Query

In [ ]:
!pip install google-cloud
!pip install google-cloud-bigquery
!pip install pandas-gbq

In [2]:
from google.cloud import bigquery
import pandas as pd
import numpy as np

In [5]:
path = "/content/drive/MyDrive/southern-ivy-378521-58bc704ea77d.json"
bqclient = bigquery.Client.from_service_account_json(path)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
sql = """
    SELECT *
    FROM `maps_reviews.reviews`
"""
result = bqclient.query(sql)
df = result.to_dataframe()

#print(df)

- Creating a new column according to the values of 'rating'
- We're going to set values for 'rating' between 1 and 3 with 0 due to sum of both values versus sum of 4 and 5 it's only the 12% of all data.
- Exporting in order to don't realize another transforms in PySpark.

In [ ]:
df.loc[df['rating'] <= 3, 'target'] = 0
df.loc[df['rating'] > 3, 'target'] = 1
df.to_csv('sentiment.csv', sep=';', index = False, encoding='utf-8')

In [ ]:
df['label'] = np.select((df['rating'] <= 3, df['rating'] > 3), [0,1], default='Not Specified')

df.to_csv('sentiment.csv', sep=';', index = False, encoding='utf-8')

# **Developing first model in Python**

In [11]:
df = pd.read_csv('/content/drive/MyDrive/sentiment.csv', sep=';')

In [12]:
df_sentiment = df[['text','rating']]
df_sentiment = df_sentiment.head(3700845) #Trabajaremos con una cuarta parte para facilitar el desarrollo del bosquejo

Some transformations

In [13]:
df_sentiment['text'] = df_sentiment['text'].astype(str)

mask=(df_sentiment['text'] == 'nan')
df_sentiment.loc[mask, 'text'] = 'sin comentario'

In [14]:
df_sentiment.loc[df['rating'] <= 3, 'target'] = 0
df_sentiment.loc[df['rating'] > 3, 'target'] = 1

Importing modules, libraries and building model

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from nltk.tokenize import RegexpTokenizer

In [17]:
token = RegexpTokenizer(r'[a-zA-Z0-9]')
cv = CountVectorizer(lowercase=True, ngram_range=(1,2), tokenizer=token.tokenize)
text_counts= cv.fit_transform(df_sentiment['text'])

In [18]:
X_train, X_test, y_train, y_test = train_test_split(text_counts, df_sentiment['target'], test_size=0.5, random_state=1)

In [19]:
clf = MultinomialNB().fit(X_train, y_train)

predicted = clf.predict(X_test)

print("MultinomialNB Accuracy:", metrics.accuracy_score(y_test, predicted))

MultinomialNB Accuracy: 0.7900312523136602


# Developing a Machine Learning Model using PySpark

In [22]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop2.tgz 
!tar xf spark-3.3.2-bin-hadoop2.tgz 
!pip install -q findspark

In [20]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop2"

In [23]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkContext
sqlSession = SparkSession.builder.master("local[*]").getOrCreate()
sc = SparkContext.getOrCreate()

In [24]:
sc.stop()

In [ ]:
#spark.sparkContext.stop()
#spark.stop()
#exit()

In [25]:
import findspark
findspark.init()
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer, StopWordsRemover

#create Spark session
appName = "Sentiment analysis in Spark"
spark = SparkSession \
    .builder \
    .appName(appName) \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [26]:
path = '/content/drive/MyDrive/sentiment.csv'
rewiews_csv = spark.read.option("delimiter", ";").option("header", "true").csv(path)
rewiews_csv.show(truncate=False, n=5)

+-------------------------------------+-------+------------+------+--------------------------------------------------------------------+-----+
|gmap_id                              |id_meta|state       |rating|text                                                                |label|
+-------------------------------------+-------+------------+------+--------------------------------------------------------------------+-----+
|0x52c8cb501767dc7b:0xf619a34f077c0cc8|360729 |North_Dakota|5     |Very nice staff that work one on one with you to find what you want.|1    |
|0x52cf94b3e465060d:0xfa49847af84d53fd|376057 |North_Dakota|5     |Servers were kind, the food was great, and the whole place clean!   |1    |
|0x52d244756c0e8cf3:0x4d2ee009c9dd3034|369512 |South_Dakota|5     |Love this place always awesome. Great food.                         |1    |
|0x533a741a38ac4e4d:0x9325dd4d13b24e27|237209 |Montana     |5     |null                                                                |1    |

Imputing null values in 'text' column

In [27]:
#colNames = {"text"}
rewiews_csv = rewiews_csv.na.fill("not commented", 'text')

In [28]:
import re, string
string.punctuation

punc = '!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~'

def sin_lower_spc(x):
    converted = x.encode('utf-8')
    lowercased_str = converted.lower()
    # for more difficult cases use re.split(' A|B')
    lowercased_str = lowercased_str.replace('--',' ')
    clean_str = lowercased_str.translate(None, punc) #Change 1
    return clean_str

In [29]:
rewiews_csv.show(truncate=False, n=5)

+-------------------------------------+-------+------------+------+--------------------------------------------------------------------+-----+
|gmap_id                              |id_meta|state       |rating|text                                                                |label|
+-------------------------------------+-------+------------+------+--------------------------------------------------------------------+-----+
|0x52c8cb501767dc7b:0xf619a34f077c0cc8|360729 |North_Dakota|5     |Very nice staff that work one on one with you to find what you want.|1    |
|0x52cf94b3e465060d:0xfa49847af84d53fd|376057 |North_Dakota|5     |Servers were kind, the food was great, and the whole place clean!   |1    |
|0x52d244756c0e8cf3:0x4d2ee009c9dd3034|369512 |South_Dakota|5     |Love this place always awesome. Great food.                         |1    |
|0x533a741a38ac4e4d:0x9325dd4d13b24e27|237209 |Montana     |5     |not commented                                                       |1    |

In [30]:
data = rewiews_csv.select('text','label')
data.show(n=5)

+--------------------+-----+
|                text|label|
+--------------------+-----+
|Very nice staff t...|    1|
|Servers were kind...|    1|
|Love this place a...|    1|
|       not commented|    1|
|       not commented|    1|
+--------------------+-----+
only showing top 5 rows



Cleaning 'text' column because it has many differents of special characters, emojis

In [31]:
from nltk.stem import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import string
import nltk

In [ ]:
palabras = [x for x in word_tokenize(text.lower()) if x not in stop_words + list(string.punctuation)]
for x in data.columns if x not in list(string.punctuation) #no lo he ensayado porque deberia ser en python

In [33]:
#tambien me funcionó
import pyspark.sql.functions as F
from pyspark.sql import types as T
import re

data = data.select([F.col(col).alias(re.sub("[^0-9a-zA-Z$]+","",col)) for col in data.columns])

In [34]:
#esta funcionó

import re

#replacing all the special characters using list comprehension
[re.sub('[\)|\(|\s|,|%]','',x) for x in data.columns]
#['id', 'id', 'id', 'ik', 'ij']

data.toDF(*[re.sub('[\)|\(|\s|,|%]','',x) for x in data.columns])
#DataFrame[id: string, id: string, id: string, ik: string, ij: string]

DataFrame[text: string, label: string]

In [35]:
#Esta tambien
data = data.withColumn(
    "text_c",
    F.regexp_replace(F.regexp_replace("text", "[^\x00-\x7F]+", ""), '""', '')
)

In [36]:
#esto esta bueno solo ejecutelo
splitData = data.randomSplit([0.6, 0.4]) 
trainingData = splitData[0] #index 0 = data training
testingData = splitData[1] #index 1 = data testing
train_rows = trainingData.count()
test_rows = testingData.count()
print ("Training data rows:", train_rows, "; Testing data rows:", test_rows)

Training data rows: 8876942 ; Testing data rows: 5926362


In [37]:
tokenizer = Tokenizer(inputCol="text", outputCol="sentimentText")
tokenizedTrain = tokenizer.transform(trainingData)
tokenizedTrain.show(truncate=False, n=5)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                                                                     

In [38]:
swr = StopWordsRemover(inputCol=tokenizer.getOutputCol(), 
                       outputCol="MeaningfulWords")
SwRemovedTrain = swr.transform(tokenizedTrain)
SwRemovedTrain.show(truncate=False, n=5)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [40]:
hashTF = HashingTF(inputCol=swr.getOutputCol(), outputCol="features")
numericTrainData = hashTF.transform(SwRemovedTrain).select(
    'label', 'MeaningfulWords', 'features')
numericTrainData.show(truncate=False, n=5)

+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|MeaningfulWords                                                                                                                                                                                                |features                                                                                                                                                                                                                                                         

In [41]:
numericTrainData

DataFrame[label: string, MeaningfulWords: array<string>, features: vector]

In [ ]:
############ Funtion in Python for Emoji removal - FACU

In [ ]:
!pip install emot #Emoji Library Installation

In [ ]:
import re
import pickle
from emot.emo_unicode import UNICODE_EMOJI, EMOTICONS_EMO

In [ ]:
import re #Funtion to remove EMOJIS
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F" # emoticons
                           u"\U0001F300-\U0001F5FF" # symbols & pictographs
                           u"\U0001F680-\U0001F6FF" # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF" # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

remove_emoji("Hilarious 😂! The feeling of making a sale 😎, The feeling of actually fulfilling orders 😒")

'Hilarious ! The feeling of making a sale , The feeling of actually fulfilling orders '

In [ ]:
df_sentiment['text'] = df_sentiment['text'].apply(remove_emoji)


In [ ]:
df_sentiment['text'].head(10)

0    Very nice staff that work one on one with you ...
1    Servers were kind, the food was great, and the...
2          Love this place always awesome. Great food.
3                                       sin comentario
4                                       sin comentario
5                       Best ribs, wings, and burgers!
6                                       sin comentario
7                         Great service and atmosphere
8                                       sin comentario
9                                       sin comentario
Name: text, dtype: object

In [ ]:
###################

In [42]:
numericTrainData =  numericTrainData.withColumn("label",col("label").cast("Integer"))

In [ ]:
numericTrainData

DataFrame[label: int, MeaningfulWords: array<string>, features: vector]

Train our classifier model using training data

In [43]:
lr = LogisticRegression(labelCol="label", featuresCol="features", 
                        maxIter=10, regParam=0.01)
model = lr.fit(numericTrainData)
print ("Training is done!")

Training is done!


Prepare testing data

In [44]:
tokenizedTest = tokenizer.transform(testingData)
SwRemovedTest = swr.transform(tokenizedTest)
numericTest = hashTF.transform(SwRemovedTest).select(
    'Label', 'MeaningfulWords', 'features')
numericTest.show(truncate=False, n=2)

+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Predict testing data and calculate the accuracy model

In [45]:
prediction = model.transform(numericTest)
predictionFinal = prediction.select(
    "MeaningfulWords", "prediction", "Label")
predictionFinal.show(n=4, truncate = False)
correctPrediction = predictionFinal.filter(
    predictionFinal['prediction'] == predictionFinal['Label']).count()
totalData = predictionFinal.count()
print("correct prediction:", correctPrediction, ", total data:", totalData, 
      ", accuracy:", correctPrediction/totalData)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+-----+
|MeaningfulWords                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |prediction|L